In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
df16 = pd.read_csv('2016.csv')
df17 = pd.read_csv('2017.csv')
df18 = pd.read_csv('2018.csv')
df19 = pd.read_csv('2019.csv')
df20 = pd.read_csv('2020.csv')
df21 = pd.read_csv('2021.csv')
df22 = pd.read_csv('2022.csv')

dfs = [df16, df17, df18, df19, df20, df21, df22]

In [4]:
for df in dfs:
    print(df.columns, df.shape)

Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (25601, 9)
Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (30177, 9)
Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (47853, 9)
Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (55285, 9)
Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (54608, 9)
Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object') (31890, 9

In [5]:
# Concatenate along rows (axis=0)
data = pd.concat(dfs, axis=0)
data.shape

(304288, 9)

In [6]:
data.isnull().sum()

Institute                    0
Academic Program Name        1
Quota                        1
Seat Type                    2
Gender                   55781
Opening Rank                 4
Closing Rank                 4
Year                         0
Round                        0
dtype: int64

In [7]:
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
for year in years:
    print(year)
    print(data[data['Year'] == year]['Gender'].value_counts(), "\n")

2016
Series([], Name: count, dtype: int64) 

2017
Series([], Name: count, dtype: int64) 

2018
Gender
Gender-Neutral                           30491
Female-only (including Supernumerary)    17362
Name: count, dtype: int64 

2019
Gender
Gender-Neutral                           35222
Female-only (including Supernumerary)    20062
Name: count, dtype: int64 

2020
Gender
Gender-Neutral                           34013
Female-only (including Supernumerary)    20595
Name: count, dtype: int64 

2021
Gender
Gender-Neutral                           17900
Female-only (including Supernumerary)    10908
Name: count, dtype: int64 

2022
Gender
Gender-Neutral                           37936
Female-only (including Supernumerary)    24017
F                                            1
Name: count, dtype: int64 



So, data for years 2016 and 2017 does not have gender mentioned, so lets keep the gender there as Gender-neutral. Aslo, there is one "F" value in Gender of 2022 data, most probably its Female but lets drop that because if it is not a female, then it will become an outlier which will screw our results.

In [8]:
data['Gender'] = data['Gender'].fillna('Gender-Neutral')
data = data.drop(data[data['Gender'] == 'F'].index)

In [9]:
data.shape

(304281, 9)

In [10]:
data.nunique()

Institute                  123
Academic Program Name      265
Quota                        7
Seat Type                   10
Gender                       2
Opening Rank             41237
Closing Rank             56271
Year                         7
Round                        7
dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304281 entries, 0 to 58873
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Institute              304281 non-null  object
 1   Academic Program Name  304280 non-null  object
 2   Quota                  304280 non-null  object
 3   Seat Type              304279 non-null  object
 4   Gender                 304281 non-null  object
 5   Opening Rank           304278 non-null  object
 6   Closing Rank           304278 non-null  object
 7   Year                   304281 non-null  int64 
 8   Round                  304281 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 23.2+ MB


In [12]:
data.isnull().sum()

Institute                0
Academic Program Name    1
Quota                    1
Seat Type                2
Gender                   0
Opening Rank             3
Closing Rank             3
Year                     0
Round                    0
dtype: int64

In [13]:
data = data.dropna()
data.shape

(304278, 9)

Why closing and opening ranks are object? There are some ranks where letter P is present for preperatory courses

In [14]:
count = 0
a = []
for i in data['Closing Rank']:
    try:
        int(i)
    except:
        count += 1
        a.append(i)
    

In [15]:
print(count, a[:20])

5589 ['119P', '50P', '50P', '64P', '197P', '3P', '7P', '159P', '15P', '1P', '19P', '45P', '150P', '4P', '238P', '580P', '1061P', '262P', '5P', '145P']


These all are Preperatory closing ranks. We will drop them.

In [16]:
data['Opening Rank'] = pd.to_numeric(data['Opening Rank'], errors='coerce')
data['Closing Rank'] = pd.to_numeric(data['Closing Rank'], errors='coerce')
data = data.dropna(subset=['Opening Rank', 'Closing Rank'])
data.shape

(298740, 9)

In [17]:
data.Quota.unique()

array(['AI', 'HS', 'OS', 'AP', 'GO', 'JK', 'LA'], dtype=object)

From the unique quota values, one thing to note that many of this quotas are applicable to NIT+ system. IITs have no such quotas, so will fiter only IIT colleges first

In [18]:
data.Institute.unique()

array(['Indian Institute of Technology Bhubaneswar',
       'Indian Institute of Technology Bombay',
       'Indian Institute of Technology Mandi',
       'Indian Institute of Technology Delhi',
       'Indian Institute of Technology Indore',
       'Indian Institute of Technology Kharagpur',
       'Indian Institute of Technology Hyderabad',
       'Indian Institute of Technology Jodhpur',
       'Indian Institute of Technology Kanpur',
       'Indian Institute of Technology Madras',
       'Indian Institute of Technology Gandhinagar',
       'Indian Institute of Technology Patna',
       'Indian Institute of Technology Roorkee',
       'Indian School of Mines Dhanbad',
       'Indian Institute of Technology Ropar',
       'Indian Institute of Technology (BHU) Varanasi',
       'Indian Institute of Technology Guwahati',
       'Indian Institute of Technology Bhilai',
       'Indian Institute of Technology Goa',
       'Indian Institute of Technology Palakkad',
       'Indian Institute

In [19]:
data_iits = data[data['Institute'].str.contains(r'^Indian Institute of Technology')]
data_iits.Institute.unique()

array(['Indian Institute of Technology Bhubaneswar',
       'Indian Institute of Technology Bombay',
       'Indian Institute of Technology Mandi',
       'Indian Institute of Technology Delhi',
       'Indian Institute of Technology Indore',
       'Indian Institute of Technology Kharagpur',
       'Indian Institute of Technology Hyderabad',
       'Indian Institute of Technology Jodhpur',
       'Indian Institute of Technology Kanpur',
       'Indian Institute of Technology Madras',
       'Indian Institute of Technology Gandhinagar',
       'Indian Institute of Technology Patna',
       'Indian Institute of Technology Roorkee',
       'Indian Institute of Technology Ropar',
       'Indian Institute of Technology (BHU) Varanasi',
       'Indian Institute of Technology Guwahati',
       'Indian Institute of Technology Bhilai',
       'Indian Institute of Technology Goa',
       'Indian Institute of Technology Palakkad',
       'Indian Institute of Technology Tirupati',
       'Indian 

In [20]:
data_iits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84083 entries, 0 to 51944
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Institute              84083 non-null  object 
 1   Academic Program Name  84083 non-null  object 
 2   Quota                  84083 non-null  object 
 3   Seat Type              84083 non-null  object 
 4   Gender                 84083 non-null  object 
 5   Opening Rank           84083 non-null  float64
 6   Closing Rank           84083 non-null  float64
 7   Year                   84083 non-null  int64  
 8   Round                  84083 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 6.4+ MB


In [21]:
data_iits.isnull().sum()

Institute                0
Academic Program Name    0
Quota                    0
Seat Type                0
Gender                   0
Opening Rank             0
Closing Rank             0
Year                     0
Round                    0
dtype: int64

In [22]:
data_iits2 = data_iits.dropna()

In [23]:
data_iits2.shape

(84083, 9)

In [24]:
data_iits2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84083 entries, 0 to 51944
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Institute              84083 non-null  object 
 1   Academic Program Name  84083 non-null  object 
 2   Quota                  84083 non-null  object 
 3   Seat Type              84083 non-null  object 
 4   Gender                 84083 non-null  object 
 5   Opening Rank           84083 non-null  float64
 6   Closing Rank           84083 non-null  float64
 7   Year                   84083 non-null  int64  
 8   Round                  84083 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 6.4+ MB


In [25]:
data_iits2.head()

,Institute,Academic Program Name,Quota,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
0,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OPEN,Gender-Neutral,3533.0,5947.0,2016,1
1,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,OBC-NCL,Gender-Neutral,1829.0,2213.0,2016,1
2,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,SC,Gender-Neutral,663.0,1023.0,2016,1
3,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",AI,ST,Gender-Neutral,331.0,357.0,2016,1
4,Indian Institute of Technology Bhubaneswar,Civil Engineering and M. Tech. in Structural E...,AI,OPEN,Gender-Neutral,5408.0,6561.0,2016,1


In [26]:
data_iits2.Quota.unique()

array(['AI'], dtype=object)

Only one unique value in Quota, so can drop it

In [27]:
data_iits3 = data_iits2.drop(['Quota'], axis = 1)

In [28]:
data_iits3

,Institute,Academic Program Name,Seat Type,Gender,Opening Rank,Closing Rank,Year,Round
0,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",OPEN,Gender-Neutral,3533.0,5947.0,2016,1
1,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",OBC-NCL,Gender-Neutral,1829.0,2213.0,2016,1
2,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",SC,Gender-Neutral,663.0,1023.0,2016,1
3,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",ST,Gender-Neutral,331.0,357.0,2016,1
4,Indian Institute of Technology Bhubaneswar,Civil Engineering and M. Tech. in Structural E...,OPEN,Gender-Neutral,5408.0,6561.0,2016,1
...,...,...,...,...,...,...,...,...
51940,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",OBC-NCL,Female-only (including Supernumerary),7459.0,8884.0,2022,6
51941,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",SC,Gender-Neutral,1709.0,2379.0,2022,6
51942,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",SC,Female-only (including Supernumerary),3122.0,3122.0,2022,6
51943,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",ST,Gender-Neutral,1044.0,1095.0,2022,6


In [29]:
max_round = []
for year in years:
    print(year)
    rounds = data_iits3[data_iits3['Year'] == year].Round.unique()
    max_round.append(max(rounds))
    print(rounds)
max_round

2016
[1 2 3 4 5 6]
2017
[1 2 3 4 5 6 7]
2018
[1 2 3 4 5 6 7]
2019
[1 2 3 4 5 6 7]
2020
[1 2 3 4 5 6]
2021
[1 2 3 5 6]
2022
[4 1 2 3 5 6]


[6, 7, 7, 7, 6, 6, 6]

In [30]:
data_iits4 = pd.DataFrame()

for year, round_ in zip(years, max_round):
    data_iits4 = pd.concat([data_iits4, data_iits3[(data_iits3['Year'] == year) & (data_iits3['Round'] == round_)]])

data_iits4 = data_iits4.drop(['Round'], axis = 1)

In [31]:
data_iits4

,Institute,Academic Program Name,Seat Type,Gender,Opening Rank,Closing Rank,Year
21375,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",OPEN,Gender-Neutral,5456.0,7066.0,2016
21376,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",OBC-NCL,Gender-Neutral,1941.0,2487.0,2016
21377,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",SC,Gender-Neutral,663.0,1178.0,2016
21378,Indian Institute of Technology Bhubaneswar,"Civil Engineering (4 Years, Bachelor of Techno...",ST,Gender-Neutral,357.0,416.0,2016
21379,Indian Institute of Technology Bhubaneswar,Civil Engineering and M. Tech. in Structural E...,OPEN,Gender-Neutral,5408.0,7295.0,2016
...,...,...,...,...,...,...,...
51940,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",OBC-NCL,Female-only (including Supernumerary),7459.0,8884.0,2022
51941,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",SC,Gender-Neutral,1709.0,2379.0,2022
51942,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",SC,Female-only (including Supernumerary),3122.0,3122.0,2022
51943,Indian Institute of Technology Dharwad,"Mechanical Engineering (4 Years, Bachelor of T...",ST,Gender-Neutral,1044.0,1095.0,2022


In [32]:
nirf = pd.read_csv('nirf.csv')

In [33]:
nirf

,Institute,Rank
0,Indian Institute of Technology Bombay,1
1,Indian Institute of Technology Delhi,2
2,Indian Institute of Technology Madras,3
3,Indian Institute of Technology Kanpur,4
4,Indian Institute of Technology Roorkee,5
5,Indian Institute of Technology Kharagpur,6
6,Indian Institute of Technology Guwahati,7
7,Indian Institute of Technology Hyderabad,8
8,Indian Institute of Technology Indore,9
9,Indian Institute of Technology (BHU) Varanasi,10


In [34]:
data_iits5 = data_iits4.copy()
data_iits5 = pd.merge(data_iits5, nirf, on='Institute', how='inner')
data_iits5 = data_iits5.rename(columns = {'Rank': 'Institute_ranked'})
data_iits5 = data_iits5.drop(['Institute'], axis = 1) 

In [35]:
data_iits5

,Academic Program Name,Seat Type,Gender,Opening Rank,Closing Rank,Year,Institute_ranked
0,"Civil Engineering (4 Years, Bachelor of Techno...",OPEN,Gender-Neutral,5456.0,7066.0,2016,17
1,"Civil Engineering (4 Years, Bachelor of Techno...",OBC-NCL,Gender-Neutral,1941.0,2487.0,2016,17
2,"Civil Engineering (4 Years, Bachelor of Techno...",SC,Gender-Neutral,663.0,1178.0,2016,17
3,"Civil Engineering (4 Years, Bachelor of Techno...",ST,Gender-Neutral,357.0,416.0,2016,17
4,Civil Engineering and M. Tech. in Structural E...,OPEN,Gender-Neutral,5408.0,7295.0,2016,17
...,...,...,...,...,...,...,...
13684,"Petroleum Engineering (4 Years, Bachelor of Te...",OBC-NCL,Female-only (including Supernumerary),7131.0,8597.0,2022,11
13685,"Petroleum Engineering (4 Years, Bachelor of Te...",SC,Gender-Neutral,1993.0,2441.0,2022,11
13686,"Petroleum Engineering (4 Years, Bachelor of Te...",SC,Female-only (including Supernumerary),3101.0,3681.0,2022,11
13687,"Petroleum Engineering (4 Years, Bachelor of Te...",ST,Gender-Neutral,812.0,1061.0,2022,11


In [36]:
Gender_labelled = pd.DataFrame({'Gender': ['Gender-Neutral', 'Female-only (including Supernumerary)'], 'label': [1,2]})
Gender_labelled

,Gender,label
0,Gender-Neutral,1
1,Female-only (including Supernumerary),2


In [37]:
data_iits6 = data_iits5.copy()
data_iits6 = pd.merge(data_iits6, Gender_labelled, on='Gender', how='inner')
data_iits6 = data_iits6.rename(columns = {'label': 'Gender_labelled'})
data_iits6 = data_iits6.drop(['Gender'], axis = 1)

In [38]:
data_iits6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13689 entries, 0 to 13688
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Academic Program Name  13689 non-null  object 
 1   Seat Type              13689 non-null  object 
 2   Opening Rank           13689 non-null  float64
 3   Closing Rank           13689 non-null  float64
 4   Year                   13689 non-null  int64  
 5   Institute_ranked       13689 non-null  int64  
 6   Gender_labelled        13689 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 748.7+ KB


In [39]:
data_iits6['Academic Program Name'].nunique()

146

In [40]:
d = data_iits6.copy()
d['Academic Program Name'][64
                          ]

'Computer Science and Engineering (4 Years, Bachelor of Technology)'

In [41]:
a = d[(d['Academic Program Name'] == 'Computer Science and Engineering (4 Years, Bachelor of Technology)') & (d['Seat Type'] == 'OBC-NCL') & (d['Institute_ranked'] == 18) & (d['Gender_labelled'] == 1)]
a

,Academic Program Name,Seat Type,Opening Rank,Closing Rank,Year,Institute_ranked,Gender_labelled
7848,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,1345.0,1593.0,2016,18,1
7868,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,1332.0,1620.0,2017,18,1
7889,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,1229.0,1774.0,2018,18,1
7912,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,984.0,1318.0,2019,18,1
7936,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,1112.0,1663.0,2020,18,1
7962,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,934.0,1329.0,2021,18,1
7986,"Computer Science and Engineering (4 Years, Bac...",OBC-NCL,1027.0,1432.0,2022,18,1


In [42]:
data_iits7 = data_iits6.copy()
data_iits7 = pd.get_dummies(data_iits7, columns=['Academic Program Name'])
data_iits7 = data_iits7.drop(['Opening Rank'], axis = 1)
data_iits7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13689 entries, 0 to 13688
Columns: 151 entries, Seat Type to Academic Program Name_Textile Technology (4 Years, Bachelor of Technology)
dtypes: bool(146), float64(1), int64(3), object(1)
memory usage: 2.4+ MB


In [43]:
seat_type = pd.DataFrame(data_iits7['Seat Type'].unique())
seat_type

,0
0,OPEN
1,OBC-NCL
2,SC
3,ST
4,OPEN (PwD)
5,OBC-NCL (PwD)
6,EWS
7,EWS (PwD)
8,SC (PwD)
9,ST (PwD)


In [44]:
data_iits7['Seat Type'].value_counts()

Seat Type
OPEN             3051
OBC-NCL          2960
SC               2847
ST               2086
EWS              1781
OPEN (PwD)        603
OBC-NCL (PwD)     266
EWS (PwD)          39
SC (PwD)           39
ST (PwD)           17
Name: count, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# def tt_split()

# def fit_ml_model(group):
#     model = LinearRegression()
#     X = group.drop('Seat Type', axis=1)  # Use all numeric columns as features
#     y = group['Closing Rank']  # The 'Category' column as the target
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     model.fit(X_train, y_train)
#     return model, X_test, y_test

# # Group by 'Category' and apply a machine learning model to each group
# seat_wise = data_iits7.groupby('Seat Type')
# for a, b in seat_wise:
#     print(a, b.columns)
    

In [46]:
values_to_remove = ['OPEN (PwD)', 'OBC-NCL (PwD)','EWS (PwD)', 'SC (PwD)', 'ST (PwD)']

# Drop rows with the specified values in 'Column_Name'
filtered_data = data_iits7.loc[~data_iits7['Seat Type'].isin(values_to_remove)]

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

data_iits8 = data_iits7.copy()
# Separate features (X) and target (y)
# X = data_iits8.drop('Seat Type', axis=1)
# y = data_iits8['Seat Type']

# Group the DataFrame by the 'Category' column
# grouped_data = data_iits8.groupby('Seat Type')
grouped_data = filtered_data.groupby('Seat Type')
group_names = list(grouped_data.groups.keys())
# Iterate through each group

models = {}

for group_name, group_df in grouped_data:
    print(f"Group: {group_name}")
    
    # Separate features and target for the current group
    X_group = group_df.drop(['Seat Type', 'Closing Rank'], axis=1)
    y_group = group_df['Closing Rank']
    
    # Split the data into training and testing sets for the current group
    X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, test_size=0.2, random_state=12)
    
    # Create and train a Linear Regression model for the current group
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Evaluate the model's performance for the current group using R-squared score
    r2 = r2_score(y_test, y_pred)
    print(model.score(X_test,y_test))
    print("R-squared Score:", r2)
    
    print("=" * 20)
    
    models[group_name] = model


Group: EWS
-6.4234719141023976e+16
R-squared Score: -6.4234719141023976e+16
Group: OBC-NCL
0.8678682825232328
R-squared Score: 0.8678682825232328
Group: OPEN
-8.080707295773378e+19
R-squared Score: -8.080707295773378e+19
Group: SC
-2.2840537403270176e+21
R-squared Score: -2.2840537403270176e+21
Group: ST
0.6416461813259862
R-squared Score: 0.6416461813259862


In [48]:
data_iits8.columns

Index(['Seat Type', 'Closing Rank', 'Year', 'Institute_ranked',
       'Gender_labelled',
       'Academic Program Name_Aerospace Engineering (4 Years, Bachelor of Technology)',
       'Academic Program Name_Aerospace Engineering (5 Years, Bachelor and Master of Technology (Dual Degree))',
       'Academic Program Name_Agricultural and Food Engineering (4 Years, Bachelor of Technology)',
       'Academic Program Name_Agricultural and Food Engineering with M.Tech. in any of the listed specializations (5 Years, Bachelor and Master of Technology (Dual Degree))',
       'Academic Program Name_Applied Geology (4 Years, Bachelor of Science)',
       ...
       'Academic Program Name_Pharmaceutics (5 Years, Bachelor and Master of Pharmaceutics(Dual Degree))',
       'Academic Program Name_Physics (4 Years, Bachelor of Science)',
       'Academic Program Name_Physics (5 Years, Bachelor of Science and Master of Science (Dual Degree))',
       'Academic Program Name_Physics (5 Years, Integrated 

In [49]:
acad = 'Academic Program Name_Aerospace Engineering (4 Years, Bachelor of Technology)'
t = [2022, 1, 1]

In [50]:
df = data_iits4.copy()

In [51]:
data.columns

Index(['Institute', 'Academic Program Name', 'Quota', 'Seat Type', 'Gender',
       'Opening Rank', 'Closing Rank', 'Year', 'Round'],
      dtype='object')

In [52]:

for column in data.columns:
    try:
        print(column, data[column].str.len().max())
    except:
        print(column, data[column].astype(str).str.len().max())

Institute 125
Academic Program Name 216
Quota 2
Seat Type 13
Gender 37
Opening Rank 9
Closing Rank 9
Year 4
Round 1


In [62]:
data.to_csv('data.csv', index = False)

In [60]:
data['Closing Rank'] = data['Closing Rank'].astype(int)

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298740 entries, 0 to 58873
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Institute              298740 non-null  object
 1   Academic Program Name  298740 non-null  object
 2   Quota                  298740 non-null  object
 3   Seat Type              298740 non-null  object
 4   Gender                 298740 non-null  object
 5   Opening Rank           298740 non-null  int32 
 6   Closing Rank           298740 non-null  int32 
 7   Year                   298740 non-null  int64 
 8   Round                  298740 non-null  int64 
dtypes: int32(2), int64(2), object(5)
memory usage: 20.5+ MB
